In [1]:
import datetime
import os
from uuid import UUID
from pprint import pprint

import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

from pot.network.node import SelfNodeInfo
from pot.network.storage import BlocksStorage, NodeStorage, TransactionStorage, NodeTrustStorage, ValidatorStorage, \
    TransactionVerifiedStorage
from pot.network.manager import NodeTrust

storage_path = os.path.realpath(os.path.join(os.path.dirname('.'), '..', 'storage'))
result_path = os.path.realpath(os.path.join(os.path.dirname('.'), '..', 'monitor', 'result'))

firsts_records = 10000

load_dotenv()

True

In [2]:
def get_self_node_info(path: str) -> UUID:
    old_value = os.getenv('STORAGE_DIR')
    os.environ["STORAGE_DIR"] = path
    self_node = SelfNodeInfo(True)
    os.environ["STORAGE_DIR"] = old_value
    return self_node.identifier


def get_info_from_blockchain(path: str) -> dict:
    storage = BlocksStorage(path)
    blocks = storage.load()
    return {
        "len": len(blocks),
        "transaction_len": sum([len(block.transactions) for block in blocks])
    }


def get_info_from_nodes(path: str) -> dict:
    storage = NodeStorage(path)
    nodes = storage.load()
    trust = NodeTrust.__new__(NodeTrust)
    trust._storage = NodeTrustStorage(path)
    # trust._storage.load()
    # print(f"Trust from {path}: {trust._storage.load()}")
    # old_value = os.getenv('STORAGE_DIR')
    # os.putenv('STORAGE_DIR', path)
    # self_node = SelfNodeInfo()
    # os.putenv('STORAGE_DIR', old_value)
    return {
        "len": len(nodes),
        #"trust": trust.get_node_trust(self_node.get_node())
        "trust": trust._storage.load()
    }


def get_info_from_validators(path: str) -> dict:
    storage = ValidatorStorage(path)
    validators = storage.load()
    return {
        "len": len(validators),
        "validators": ','.join([v.hex for v in validators])
    }


def get_info_from_transactions_to_verify(path: str) -> dict:
    storage = TransactionStorage(path)
    try:
        txs = storage.load()
    except Exception as e:
        print(f"Error while processing path {path}: {e}")
        txs = []
    return {
        "len": len(txs)
    }


def get_info_from_transactions_verified(path: str) -> dict:
    storage = TransactionVerifiedStorage(path)
    txs = storage.load()
    return {
        "len": len(txs)
    }

In [3]:
cols_dict = {
    'time': 'Time',
    'number_of_nodes': 'Number of nodes',
    'number_of_validators': 'Number of validators',
    # 'node_trust': 'Node trust',
    'number_of_blocks': 'Number of blocks',
    'number_of_transaction_to_verify': 'Number of transactions to verify',
    'number_of_verified_transactions': 'Number of verified transactions',
    'validators': 'Validators'
}

cols = list(cols_dict.keys())

df_trust = pd.DataFrame(columns=["time", "sourceNode", "node", "trust"])
df_trust.set_index(["time", "sourceNode", "node"], inplace=True)
# df_trust.sort_index()

In [4]:
first_time = None
dfs = {}

nodes_mapping = {}

for node in os.listdir(storage_path):
    # list all nodes in dir

    if node == '.gitignore':
        continue

    print(f"Processing node {node}")

    trust_data = []

    df = pd.DataFrame(columns=cols)
    df.set_index('time', inplace=True)

    dirs = [int(time_dir) for time_dir in os.listdir(os.path.join(storage_path, node, 'dump'))]
    dirs.sort()
    print(f"Found {len(dirs)} dirs in node {node}")

    record_i = 0
    for time in dirs:
        # list all time in dir
        if not first_time:
            first_time = time

        if firsts_records is not None and firsts_records < record_i:
            break

        storage_dir = os.path.join(storage_path, node, 'dump', str(time))
        # print(f"Processing dir {storage_dir}")

        if node not in nodes_mapping.keys():
            self_node_id = get_self_node_info(storage_dir)
            nodes_mapping[node] = self_node_id.hex
            print(self_node_id.hex)

        try:
            #self_node_id = get_self_node_info(storage_dir)
            blocks_info = get_info_from_blockchain(storage_dir)
            nodes_info = get_info_from_nodes(storage_dir)
            validators_info = get_info_from_validators(storage_dir)
            txs_to_ver_info = get_info_from_transactions_to_verify(storage_dir)
            txs_ver_info = get_info_from_transactions_verified(storage_dir)
        except Exception as e:
            print(f"Exception {e} while loading from node: {node} from time {time}")
            raise e

        actual_time = time - first_time
        df.loc[actual_time] = [
            nodes_info['len'],
            validators_info['len'],
            # nodes_info["trust"],
            blocks_info['len'],
            txs_to_ver_info['len'],
            txs_ver_info['len'],
            validators_info["validators"]
        ]
        for node_id, trust in nodes_info["trust"].items():
            trust_data.append({
                "time": actual_time,
                "sourceNode": node,
                "node": node_id.hex,
                "trust": trust
            })
            # df_trust.loc[(actual_time, node, node_id.hex), :] = trust
        record_i += 1

    # check if all df has step by step info
    #df.to_excel(os.path.join(result_path, f"result-{node}.xlsx"))
    dfs[node] = df

    df_trust_node = pd.DataFrame(trust_data, columns=["time", "sourceNode", "node", "trust"])
    df_trust_node.set_index(["time", "sourceNode", "node"], inplace=True)
    #df_trust_node.to_excel(os.path.join(result_path, f"result-trust-{node}.xlsx"))

    df_trust = pd.concat([df_trust, df_trust_node])


Processing node e5ef6f3d6292
Found 1859 dirs in node e5ef6f3d6292
ab086f26851a46538805ad83d144fad6
Processing node 0a479aec6c60
Found 1880 dirs in node 0a479aec6c60
9b55ff3ca9b540778da0ff576f69cc10
Processing node b2af3cbba21d
Found 1889 dirs in node b2af3cbba21d
6c0d6727453640048d0ad5a977ac4ac6
Processing node 467bb1befdd6
Found 1863 dirs in node 467bb1befdd6
285b9ed67a214329af06e6a9ee5d28b2
Processing node cc2afc66a825
Found 1852 dirs in node cc2afc66a825
b8d9a3b6c6da4e2daa907687cfe4c978
Processing node 1af44b40863c
Found 1875 dirs in node 1af44b40863c
1592bdbb409f46d6ad0c1aa8e51b0360
Processing node 435c733f0ed0
Found 1868 dirs in node 435c733f0ed0
4d5cf72eb0a84162a85c26a5d0051079


In [5]:
uno = dfs.get(list(dfs.keys())[0])
uno.head()

,number_of_nodes,number_of_validators,number_of_blocks,number_of_transaction_to_verify,number_of_verified_transactions,validators
time,,,,,,
0,7,1,1,0,0,6c0d6727453640048d0ad5a977ac4ac6
1,7,1,1,0,0,6c0d6727453640048d0ad5a977ac4ac6
2,7,1,1,0,0,6c0d6727453640048d0ad5a977ac4ac6
3,7,1,1,0,0,6c0d6727453640048d0ad5a977ac4ac6
4,7,1,1,0,0,6c0d6727453640048d0ad5a977ac4ac6


In [6]:
uno["validators"].unique()

array(['6c0d6727453640048d0ad5a977ac4ac6',
       '1592bdbb409f46d6ad0c1aa8e51b0360,285b9ed67a214329af06e6a9ee5d28b2',
       '6c0d6727453640048d0ad5a977ac4ac6,9b55ff3ca9b540778da0ff576f69cc10',
       '4d5cf72eb0a84162a85c26a5d0051079,b8d9a3b6c6da4e2daa907687cfe4c978',
       'ab086f26851a46538805ad83d144fad6,4d5cf72eb0a84162a85c26a5d0051079'],
      dtype=object)

In [7]:
uno.last_valid_index()

1866

In [8]:
for validators in uno["validators"].unique():
    index_min = uno[uno["validators"] == validators].first_valid_index()
    print(f"Index {index_min} for validators {validators}")

Index 0 for validators 6c0d6727453640048d0ad5a977ac4ac6
Index 19 for validators 1592bdbb409f46d6ad0c1aa8e51b0360,285b9ed67a214329af06e6a9ee5d28b2
Index 175 for validators 6c0d6727453640048d0ad5a977ac4ac6,9b55ff3ca9b540778da0ff576f69cc10
Index 328 for validators 4d5cf72eb0a84162a85c26a5d0051079,b8d9a3b6c6da4e2daa907687cfe4c978
Index 485 for validators ab086f26851a46538805ad83d144fad6,4d5cf72eb0a84162a85c26a5d0051079
